In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import math

In [2]:
anime = pd.read_csv("data/anime.csv")
anime.head()

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009.0,2010.0,Spring,The foundation of alchemy is based on the law ...,['Bones'],"['Action', 'Adventure', 'Drama', 'Fantasy', 'M...","['Animal Abuse', 'Mature Themes', 'Violence', ...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016.0,2016.0,NaN,Mitsuha and Taki are two total strangers livin...,['CoMix Wave Films'],"['Drama', 'Romance', 'Body Swapping', 'Gender ...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016.0,2016.0,NaN,"After transferring into a new school, a deaf g...",['Kyoto Animation'],"['Drama', 'Shounen', 'Disability', 'Melancholy...","['Bullying', 'Mature Themes', 'Suicide']",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016.0,2016.0,Fall,"Picking up where the second season ended, the ...",['Production I.G'],"['Shounen', 'Sports', 'Animeism', 'School Club...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019.0,2019.0,Spring,The battle to retake Wall Maria begins now! Wi...,['Wit Studio'],"['Action', 'Fantasy', 'Horror', 'Shounen', 'Da...","['Cannibalism', 'Explicit Violence']",21308.0,3217,7864,174,4.650,15789.0


In [3]:
#Clean studios column from "list of strings" to a string
anime['studios'] = anime['studios'].str.replace("[", "")
anime['studios'] = anime['studios'].str.replace("]", "")
anime['studios'] = anime['studios'].str.lstrip("'").str.rstrip("'")
#final_username = username.replace("_", "")


In [4]:
#Clean list function
def clean_list(col):
    col = col.str.replace("[", "")
    col = col.str.replace("]", "")
    col = col.str.replace("'", "")
    col = col.str.split(",")
    return col

anime['tags'] = clean_list(anime['tags'])
anime['contentWarn'] = clean_list(anime['contentWarn'])



# anime['tags'] = anime['tags'].str.replace("[", "")
# anime['tags'] = anime['tags'].str.replace("]", "")
# anime['tags'] = anime['tags'].str.replace("'", "")
# anime['tags'] = anime['tags'].str.split(",")

In [5]:
#Change year to int instead of double to remove decimal
anime["startYr"] = anime["startYr"].fillna(0.0).astype(int)
anime["finishYr"] = anime["finishYr"].fillna(0.0).astype(int)
anime.head()

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009,2010,Spring,The foundation of alchemy is based on the law ...,Bones,"[Action, Adventure, Drama, Fantasy, Myster...","[Animal Abuse, Mature Themes, Violence, Dom...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016,2016,NaN,Mitsuha and Taki are two total strangers livin...,CoMix Wave Films,"[Drama, Romance, Body Swapping, Gender Bend...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016,2016,NaN,"After transferring into a new school, a deaf g...",Kyoto Animation,"[Drama, Shounen, Disability, Melancholy, M...","[Bullying, Mature Themes, Suicide]",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016,2016,Fall,"Picking up where the second season ended, the ...",Production I.G,"[Shounen, Sports, Animeism, School Club, S...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019,2019,Spring,The battle to retake Wall Maria begins now! Wi...,Wit Studio,"[Action, Fantasy, Horror, Shounen, Dark Fa...","[Cannibalism, Explicit Violence]",21308.0,3217,7864,174,4.650,15789.0


In [6]:
anime.sort_values(['rating'], ascending=False)

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009,2010,Spring,The foundation of alchemy is based on the law ...,Bones,"[Action, Adventure, Drama, Fantasy, Myster...","[Animal Abuse, Mature Themes, Violence, Dom...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016,2016,NaN,Mitsuha and Taki are two total strangers livin...,CoMix Wave Films,"[Drama, Romance, Body Swapping, Gender Bend...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016,2016,NaN,"After transferring into a new school, a deaf g...",Kyoto Animation,"[Drama, Shounen, Disability, Melancholy, M...","[Bullying, Mature Themes, Suicide]",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016,2016,Fall,"Picking up where the second season ended, the ...",Production I.G,"[Shounen, Sports, Animeism, School Club, S...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019,2019,Spring,The battle to retake Wall Maria begins now! Wi...,Wit Studio,"[Action, Fantasy, Horror, Shounen, Dark Fa...","[Cannibalism, Explicit Violence]",21308.0,3217,7864,174,4.650,15789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14573,"Welcome to Demon School, Iruma-kun 2",TV,NaN,NaN,False,2021,2021,NaN,"Second Season of Welcome to Demon School, Irum...",,"[Comedy, Shounen, Demons, Monster School, ...",[],0.0,0,1106,0,NaN,NaN
14574,Pinocchio-P: Sekai wa Mada Hajimatte Sura Inai,Music Video,1.0,4.0,False,2020,2020,NaN,NaN,,"[Chibi, Vocaloid]",[],9.0,1,7,1,NaN,NaN
14575,Minagoroshi,OVA,1.0,1.0,False,2005,2005,NaN,NaN,,"[Comedy, Ecchi, No Dialogue, Shorts]",[],3.0,0,7,0,NaN,NaN
14576,Kurayukaba,Movie,NaN,NaN,False,0,0,NaN,NaN,Makaria,[],[],0.0,0,62,0,NaN,NaN


In [7]:
anime['mediaType'].value_counts()

TV             4510
Movie          2512
OVA            2097
Music Video    1596
Web            1555
DVD Special     849
Other           782
TV Special      609
Name: mediaType, dtype: int64

In [8]:
tv_anime = anime.loc[anime['mediaType'] == 'TV'].sort_values('rating', ascending=False)
movie_anime = anime.loc[anime['mediaType'] == 'Movie'].sort_values('rating', ascending=False)
ova_anime = anime.loc[anime['mediaType'] == 'OVA'].sort_values('rating', ascending=False)

In [9]:
tv_anime['sznOfRelease'].value_counts()

Spring    1099
Fall       957
Winter     665
Summer     630
Name: sznOfRelease, dtype: int64

In [10]:
tv_anime['sznOfRelease'].head(100).value_counts()

Spring    31
Fall      30
Winter    23
Summer    16
Name: sznOfRelease, dtype: int64

In [13]:
# count = 0
# for index, col in tv_anime['tags'].iteritems():
#     if 'Action' in col:
#         print(col)
#         print()
#         count +=1
# count    

#Create a function that can count the number per genre
def genreIndex(genre, df):
    count = 0
    indeces = []
    for index, col in df['tags'].iteritems():
        if genre in col:
            indeces.append(index)
#             print(col)
#             print()
            count+=1
    print("There are " + str(count) + " animes")
    return indeces

#Can get total count by getting len(genreIndex)

In [23]:
genreIndex('Romance', tv_anime)
for index in genreIndex('Romance', tv_anime):
    print(tv_anime.iloc[index])
#tv_anime.iloc[0]


There are 35 animes
There are 35 animes
title                         You're Under Arrest! Fast & Furious
mediaType                                                      TV
eps                                                            26
duration                                                      NaN
ongoing                                                     False
startYr                                                      2001
finishYr                                                     2001
sznOfRelease                                               Spring
description     Life goes on and it's business as usual at Bok...
studios                                               Studio Deen
tags            [Action,  Comedy,  Drama,  Seinen,  Coworkers,...
contentWarn                                                    []
watched                                                      1572
watching                                                       67
wantWatch                           

IndexError: single positional indexer is out-of-bounds

In [24]:
#Get all the unique genres and the number of animes per genre
genres = []
genreTotal = {}
for index, col in tv_anime['tags'].iteritems():
    for genre in col:
        if (genre not in genres):
            genres.append(genre)
            genreTotal[genre] = 1

        genreTotal[genre] += 1           
#print(genreTotal)

559
559


In [16]:
#Get the top 20 genres and the total count of anime tv shows
genre_df = pd.DataFrame(list(genreTotal.items()), columns=['Genre', 'Count'])
top20 = genre_df.sort_values('Count', ascending=False).head(20)
top20.reset_index(drop=True, inplace=True)
top20

,Genre,Count
0,Based on a Manga,1655
1,Action,1297
2,Comedy,1284
3,Sci Fi,770
4,Fantasy,756
5,Original Work,728
6,Short Episodes,718
7,School Life,697
8,Shounen,685
9,Family Friendly,675


In [ ]:
#Find average rating per genre
